# NewsBot Intelligence System 2.0
Final Integrated Notebook

## Project Overview
This notebook represents the upgraded final NewsBot system.

In [1]:
# ==========================================
# CELL 1: SETUP & LIBRARIES
# ==========================================
# Installing necessary NLP libraries for the Final Project
!pip install -q spacy textblob gensim scikit-learn pandas numpy matplotlib seaborn nltk
!python -m spacy download en_core_web_sm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
import nltk
from textblob import TextBlob
from collections import defaultdict, Counter
import warnings
import random
from datetime import datetime, timedelta

# Sklearn for Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# NLTK setup
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

warnings.filterwarnings('ignore')
nlp = spacy.load('en_core_web_sm')

print("✅ NewsBot 2.0 Environment Ready")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 109.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ NewsBot 2.0 Environment Ready


In [2]:
# ==========================================
# CELL 2: CONFIGURATION & SYSTEM ARCHITECTURE
# ==========================================

class NewsBot2Config:
    """Configuration management for NewsBot 2.0"""
    def __init__(self):
        self.DATA_PATH = 'BBC News Train.csv'
        self.RANDOM_STATE = 42
        self.MAX_FEATURES = 5000
        self.NUM_TOPICS = 5
        self.CONFIDENCE_THRESHOLD = 0.6
        # Simulating dates since dataset doesn't have them
        self.SIMULATE_DATES = True

class NewsBot2System:
    """Main Orchestrator Class"""
    def __init__(self, config):
        self.config = config
        self.classifier = None
        self.topic_engine = None
        self.sentiment_engine = None
        self.ner_engine = None
        self.summarizer = None
        self.conversation = None
        self.df = None

    def initialize_modules(self):
        print("🚀 Initializing NewsBot 2.0 Modules...")
        self.classifier = AdvancedNewsClassifier(self.config)
        self.topic_engine = TopicDiscoveryEngine(self.config)
        self.sentiment_engine = SentimentEvolutionTracker()
        self.ner_engine = EntityRelationshipMapper()
        self.summarizer = IntelligentSummarizer()
        self.conversation = ConversationalInterface(self)
        print("✅ All Modules Loaded.")

    def load_and_prep_data(self):
        print("📂 Loading Data...")
        try:
            self.df = pd.read_csv(self.config.DATA_PATH)

            # Module A Preprocessing Requirement: Clean Text
            self.df['clean_text'] = self.df['Text'].apply(self._preprocess_text)

            # Simulate dates for Temporal Analysis (Module A requirement)
            if self.config.SIMULATE_DATES:
                start_date = datetime.now() - timedelta(days=365)
                self.df['date'] = [start_date + timedelta(days=random.randint(0, 365)) for _ in range(len(self.df))]

            print(f"✅ Data Loaded: {self.df.shape[0]} articles.")
        except FileNotFoundError:
            print("❌ Error: 'BBC News Train.csv' not found. Please upload the dataset.")

    def _preprocess_text(self, text):
        # Basic cleaning pipeline
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text

    def train_systems(self):
        if self.df is None: return
        print("⚙️ Training Core Models...")
        # Train Classifier
        self.classifier.train(self.df['clean_text'], self.df['Category'])
        # Train Topic Model
        self.topic_engine.fit_topics(self.df['clean_text'])
        print("✅ Training Complete.")

In [3]:
# ==========================================
# CELL 3: MODULE A - ADVANCED CONTENT ANALYSIS
# ==========================================

class AdvancedNewsClassifier:
    """
    Module A1: Enhanced Classification with Confidence Scoring
    """
    def __init__(self, config):
        self.model = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=config.MAX_FEATURES, stop_words='english')),
            ('clf', LogisticRegression(random_state=config.RANDOM_STATE, max_iter=1000, multi_class='ovr'))
        ])

    def train(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)
        preds = self.model.predict(X_test)
        print(f"   Classifier Accuracy: {accuracy_score(y_test, preds):.2f}")

    def predict_with_confidence(self, text):
        """Returns category and confidence score"""
        clean_text = [text] # Preprocessing should happen before this or included here
        probs = self.model.predict_proba(clean_text)[0]
        max_index = np.argmax(probs)
        category = self.model.classes_[max_index]
        confidence = probs[max_index]
        return category, confidence

class TopicDiscoveryEngine:
    """
    Module A2: Topic Modeling (LDA)
    """
    def __init__(self, config):
        self.vectorizer = CountVectorizer(max_features=config.MAX_FEATURES, stop_words='english')
        self.lda = LatentDirichletAllocation(n_components=config.NUM_TOPICS, random_state=config.RANDOM_STATE)

    def fit_topics(self, documents):
        self.dtm = self.vectorizer.fit_transform(documents)
        self.lda.fit(self.dtm)

    def get_article_topics(self, text):
        vec = self.vectorizer.transform([text])
        topic_dist = self.lda.transform(vec)[0]
        dominant_topic = np.argmax(topic_dist)
        return dominant_topic, topic_dist

class SentimentEvolutionTracker:
    """
    Module A3: Sentiment Analysis
    """
    def analyze_sentiment(self, text):
        blob = TextBlob(text)
        # Returns polarity (-1 to 1) and subjectivity (0 to 1)
        return {
            'polarity': blob.sentiment.polarity,
            'subjectivity': blob.sentiment.subjectivity,
            'label': 'Positive' if blob.sentiment.polarity > 0.05 else 'Negative' if blob.sentiment.polarity < -0.05 else 'Neutral'
        }

class EntityRelationshipMapper:
    """
    Module A4: NER Extraction
    """
    def extract_entities(self, text):
        doc = nlp(text[:100000]) # Limit length for performance
        entities = defaultdict(list)
        for ent in doc.ents:
            entities[ent.label_].append(ent.text)
        return dict(entities)

In [4]:
# ==========================================
# CELL 4: MODULE B - LANGUAGE UNDERSTANDING
# ==========================================

class IntelligentSummarizer:
    """
    Module B: Text Summarization (Extractive approach for efficiency)
    """
    def summarize_article(self, text, num_sentences=3):
        sentences = sent_tokenize(text)
        if len(sentences) <= num_sentences:
            return text

        # Basic frequency-based ranking
        stop_words = set(stopwords.words("english"))
        words = nltk.word_tokenize(text.lower())
        freq_table = dict()
        for word in words:
            if word not in stop_words and word.isalnum():
                freq_table[word] = freq_table.get(word, 0) + 1

        sentence_scores = dict()
        for sentence in sentences:
            for word in nltk.word_tokenize(sentence.lower()):
                if word in freq_table:
                    if len(sentence.split(' ')) < 30: # Constraint on sentence length
                        sentence_scores[sentence] = sentence_scores.get(sentence, 0) + freq_table[word]

        import heapq
        summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
        return ' '.join(summary_sentences)

In [5]:
# ==========================================
# CELL 5: MODULE D - CONVERSATIONAL INTERFACE
# ==========================================

class ConversationalInterface:
    """
    Module D: Natural Language Query Processing
    """
    def __init__(self, system):
        self.system = system

    def process_query(self, query):
        query = query.lower()

        # Intent: Analyze a specific article (Simulation)
        if "analyze this" in query or "analyze article" in query:
             return "Please provide the article text using the analyze_article() function directly."

        # Intent: Search/Filter
        if "show me" in query or "find" in query:
            if "sport" in query:
                return self._get_latest_news("sport")
            elif "tech" in query:
                return self._get_latest_news("tech")
            elif "business" in query:
                return self._get_latest_news("business")

        # Intent: Help
        return "I can help you analyze news. Try asking 'Show me tech news' or use the analyze_article method."

    def _get_latest_news(self, category):
        # Retrieve random article from category
        subset = self.system.df[self.system.df['Category'] == category]
        if subset.empty:
            return f"No news found for {category}."
        article = subset.iloc[0]
        return f"Found {category} article: {article['Text'][:100]}..."

In [6]:
# ==========================================
# CELL 6: INTEGRATION & DEMO
# ==========================================

# 1. Initialize Configuration
config = NewsBot2Config()

# 2. Initialize System
newsbot = NewsBot2System(config)

# 3. Load Data & Train
newsbot.initialize_modules()
newsbot.load_and_prep_data()
newsbot.train_systems()

# ==========================================
# DEMO: SIMULATING THE FINAL PROJECT OUTPUT
# ==========================================
print("\n" + "="*50)
print("🤖 NEWSBOT 2.0 LIVE DEMONSTRATION")
print("="*50)

# Pick a random article from the dataset to analyze
if newsbot.df is not None:
    sample_row = newsbot.df.sample(1).iloc[0]
    sample_text = sample_row['Text']
    actual_cat = sample_row['Category']

    print(f"\n📄 Analyzing Random Article (Actual ID: {sample_row['ArticleId']})")
    print(f"📝 Excerpt: {sample_text[:150]}...")

    # 1. CLASSIFICATION
    pred_cat, conf = newsbot.classifier.predict_with_confidence(sample_text)
    print(f"\n🔍 Classification Results:")
    print(f"   - Predicted Category: {pred_cat.upper()}")
    print(f"   - Confidence Score: {conf:.4f}")
    print(f"   - Actual Category: {actual_cat.upper()}")

    # 2. TOPIC MODELING
    topic_id, dist = newsbot.topic_engine.get_article_topics(sample_text)
    print(f"\n📊 Topic Analysis:")
    print(f"   - Dominant Topic ID: {topic_id}")

    # 3. SENTIMENT
    sentiment = newsbot.sentiment_engine.analyze_sentiment(sample_text)
    print(f"\n❤️ Sentiment Analysis:")
    print(f"   - Tone: {sentiment['label']}")
    print(f"   - Polarity: {sentiment['polarity']:.2f}")

    # 4. NER
    entities = newsbot.ner_engine.extract_entities(sample_text)
    print(f"\n🕸️ Entity Extraction (Top 3 per type):")
    for label, ents in list(entities.items())[:3]:
        print(f"   - {label}: {list(set(ents))[:3]}")

    # 5. SUMMARIZATION
    summary = newsbot.summarizer.summarize_article(sample_text)
    print(f"\n📝 Intelligent Summary:")
    print(f"   \"{summary}\"")

    # 6. CONVERSATION INTERFACE
    print(f"\n💬 Conversational Interface Test:")
    user_query = "Show me tech news"
    print(f"   User: {user_query}")
    print(f"   NewsBot: {newsbot.conversation.process_query(user_query)}")

else:
    print("⚠️ Please ensure the dataset is loaded to run the demo.")

🚀 Initializing NewsBot 2.0 Modules...
✅ All Modules Loaded.
📂 Loading Data...
❌ Error: 'BBC News Train.csv' not found. Please upload the dataset.

🤖 NEWSBOT 2.0 LIVE DEMONSTRATION
⚠️ Please ensure the dataset is loaded to run the demo.


## Sample Data

In [7]:

data = {
    "text": [
        "AI is transforming cybersecurity and threat detection.",
        "Global markets react to new economic policies.",
        "New climate initiatives spark political debate."
    ]
}
df = pd.DataFrame(data)
df


,text
0,AI is transforming cybersecurity and threat de...
1,Global markets react to new economic policies.
2,New climate initiatives spark political debate.


## Preprocessing

In [8]:
import re, string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)

df["clean_text"] = df["text"].apply(preprocess_text)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,clean_text
0,AI is transforming cybersecurity and threat de...,ai transforming cybersecurity threat detection
1,Global markets react to new economic policies.,global markets react new economic policies
2,New climate initiatives spark political debate.,new climate initiatives spark political debate
